# Setting Everything Up

In [2]:
import pandas as pd
from datetime import datetime

In [3]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 
from sklearn.feature_extraction.text import CountVectorizer
import json
from flatten_json import flatten


# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
# pd.options.display.max_colwidth = None

# pd.options.display.float_format = '{:,.0f}'.format

import string

%matplotlib inline

In [4]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [5]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
%%R

# My commonly used R imports

require('tidyverse')

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [7]:
#To Download PDFs
from urllib.parse import urljoin

In [8]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [9]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)
library(reshape2)
library(data.table)


R[write to console]: Loading required package: RColorBrewer

R[write to console]: Loading required package: NLP

R[write to console]: 
Attaching package: ‘NLP’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    annotate


R[write to console]: 
Attaching package: ‘reshape2’


R[write to console]: The following object is masked from ‘package:tidyr’:

    smiths


R[write to console]: data.table 1.14.8 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: **********
This installation of data.table has not detected OpenMP support. It should still work but in single-threaded mode.
This is a Mac. Please read https://mac.r-project.org/openmp/. Please engage with Apple and ask them for support. Check r-datatable.com for updates, and our Mac instructions here: https://github.com/Rdatatable/data.table/wiki/Installation. After several years of many reports of installation problems on Mac, it's time to gingerly point out that there

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
%%R
library(vistime)
library(shiny)
library(plotly)
#for timeline plotting

R[write to console]: 
Attaching package: ‘plotly’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    last_plot


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: The following object is masked from ‘package:graphics’:

    layout




In [12]:
from collections import Counter


# For Employment and age data

### Reading the Data

In [27]:
df_edu_employment_pre_separation=pd.read_csv("saf_edu_age_employment.csv")

In [28]:
df_edu_employment_pre_separation

,employment_titlea,Broad_Age_Groups_slash_County,Broad_Age_Groups_slash_County_original,Total,Working_and_Seeking_Work_slash_No_Work_Available,Working,Seeking_Work_slash_No_Work_Available,Persons_outside_the_Labour_Force,Not_Stated
0,KENYA,Total,KENYA,"41,235,190","19,677,401 2,621,158",NaN,NaN,"18,927,688","8,943"
1,KENYA,5 - 14,5 - 14,"12,505,987","969,868 -",NaN,NaN,"11,534,179","1,940"
2,KENYA,15 - 17,15 - 17,"3,360,980","379,497 127,428",NaN,NaN,"2,853,492",563
3,KENYA,18 - 24,18 - 24,"6,288,095","2,891,426 708,167",NaN,NaN,"2,687,184","1,318"
4,KENYA,25 - 34,25 - 34,"7,333,397","5,544,992 939,317",NaN,NaN,"847,376","1,712"
...,...,...,...,...,...,...,...,...,...
379,NAIROBI CITY,18 - 24,18 - 24,"701,281","279,225 123,232",NaN,NaN,"298,554",270
380,NAIROBI CITY,25 - 34,25 - 34,"1,062,332","728,095 175,830",NaN,NaN,"157,973",434
381,NAIROBI CITY,35 - 64,35 - 64,"989,479","763,332 114,150",NaN,NaN,"111,563",434
382,NAIROBI CITY,65 +,65 +,"56,503","25,413 3,281",NaN,NaN,"27,790",19


# Separating the Data

In [29]:
def get_working(x):
    try:
        working = x.split(' ')[0]
        return working
    except:
        pass

df_edu_employment_pre_separation['Working']=df_edu_employment_pre_separation['Working_and_Seeking_Work_slash_No_Work_Available'].apply(get_working)

df_edu_employment_pre_separation.head(10)

,employment_titlea,Broad_Age_Groups_slash_County,Broad_Age_Groups_slash_County_original,Total,Working_and_Seeking_Work_slash_No_Work_Available,Working,Seeking_Work_slash_No_Work_Available,Persons_outside_the_Labour_Force,Not_Stated
0,KENYA,Total,KENYA,"41,235,190","19,677,401 2,621,158","19,677,401",NaN,"18,927,688","8,943"
1,KENYA,5 - 14,5 - 14,"12,505,987","969,868 -","969,868",NaN,"11,534,179","1,940"
2,KENYA,15 - 17,15 - 17,"3,360,980","379,497 127,428","379,497",NaN,"2,853,492",563
3,KENYA,18 - 24,18 - 24,"6,288,095","2,891,426 708,167","2,891,426",NaN,"2,687,184","1,318"
4,KENYA,25 - 34,25 - 34,"7,333,397","5,544,992 939,317","5,544,992",NaN,"847,376","1,712"
5,KENYA,35 - 64,35 - 64,"9,886,139","8,371,755 791,492","8,371,755",NaN,"720,494","2,398"
6,KENYA,65 +,65 +,"1,860,021","1,519,863 54,754","1,519,863",NaN,"284,963",441
7,KENYA,Not Stated,Not Stated,571,- -,-,NaN,-,571
8,MOMBASA,Total,MOMBASA,"1,043,603","423,439 141,942","423,439",NaN,"477,965",257
9,MOMBASA,5 - 14,5 - 14,"243,191","3,289 -","3,289",NaN,"239,864",38


In [30]:
def get_seeking(x):
    try:
        seeking = x.split(' ')[-1]
        return seeking
    except:
        pass

df_edu_employment_pre_separation['Seeking_Work_slash_No_Work_Available']=df_edu_employment_pre_separation['Working_and_Seeking_Work_slash_No_Work_Available'].apply(get_seeking)

df_edu_employment_pre_separation.head(20)

,employment_titlea,Broad_Age_Groups_slash_County,Broad_Age_Groups_slash_County_original,Total,Working_and_Seeking_Work_slash_No_Work_Available,Working,Seeking_Work_slash_No_Work_Available,Persons_outside_the_Labour_Force,Not_Stated
0,KENYA,Total,KENYA,"41,235,190","19,677,401 2,621,158","19,677,401","2,621,158","18,927,688","8,943"
1,KENYA,5 - 14,5 - 14,"12,505,987","969,868 -","969,868",-,"11,534,179","1,940"
2,KENYA,15 - 17,15 - 17,"3,360,980","379,497 127,428","379,497","127,428","2,853,492",563
3,KENYA,18 - 24,18 - 24,"6,288,095","2,891,426 708,167","2,891,426","708,167","2,687,184","1,318"
4,KENYA,25 - 34,25 - 34,"7,333,397","5,544,992 939,317","5,544,992","939,317","847,376","1,712"
5,KENYA,35 - 64,35 - 64,"9,886,139","8,371,755 791,492","8,371,755","791,492","720,494","2,398"
6,KENYA,65 +,65 +,"1,860,021","1,519,863 54,754","1,519,863","54,754","284,963",441
7,KENYA,Not Stated,Not Stated,571,- -,-,-,-,571
8,MOMBASA,Total,MOMBASA,"1,043,603","423,439 141,942","423,439","141,942","477,965",257
9,MOMBASA,5 - 14,5 - 14,"243,191","3,289 -","3,289",-,"239,864",38


In [31]:
original_df=df_edu_employment_pre_separation

# Get the column titles

In [32]:
original_df

,employment_titlea,Broad_Age_Groups_slash_County,Broad_Age_Groups_slash_County_original,Total,Working_and_Seeking_Work_slash_No_Work_Available,Working,Seeking_Work_slash_No_Work_Available,Persons_outside_the_Labour_Force,Not_Stated
0,KENYA,Total,KENYA,"41,235,190","19,677,401 2,621,158","19,677,401","2,621,158","18,927,688","8,943"
1,KENYA,5 - 14,5 - 14,"12,505,987","969,868 -","969,868",-,"11,534,179","1,940"
2,KENYA,15 - 17,15 - 17,"3,360,980","379,497 127,428","379,497","127,428","2,853,492",563
3,KENYA,18 - 24,18 - 24,"6,288,095","2,891,426 708,167","2,891,426","708,167","2,687,184","1,318"
4,KENYA,25 - 34,25 - 34,"7,333,397","5,544,992 939,317","5,544,992","939,317","847,376","1,712"
...,...,...,...,...,...,...,...,...,...
379,NAIROBI CITY,18 - 24,18 - 24,"701,281","279,225 123,232","279,225","123,232","298,554",270
380,NAIROBI CITY,25 - 34,25 - 34,"1,062,332","728,095 175,830","728,095","175,830","157,973",434
381,NAIROBI CITY,35 - 64,35 - 64,"989,479","763,332 114,150","763,332","114,150","111,563",434
382,NAIROBI CITY,65 +,65 +,"56,503","25,413 3,281","25,413","3,281","27,790",19


In [33]:
import pandas as pd

# Assuming you have a DataFrame named 'df'
column_names = original_df.columns.tolist()

# Print the list of column names
print(column_names)


['employment_titlea', 'Broad_Age_Groups_slash_County', 'Broad_Age_Groups_slash_County_original', 'Total', 'Working_and_Seeking_Work_slash_No_Work_Available', 'Working', 'Seeking_Work_slash_No_Work_Available', 'Persons_outside_the_Labour_Force', 'Not_Stated']


### Setting the Data so that the all details for one county are in one row

In [34]:
import pandas as pd

# Assuming your original dataset is stored in a DataFrame called 'original_df'
# and the county names are stored in a column called 'employment_titlea'
# and the age group details are stored in a column called 'Broad_Age_Groups_slash_County'

# Define the age groups
age_groups = {
    'Total': 'Total',
    '5 - 14': '5to14',
    '15 - 17': '15to17',
    '18 - 24': '18to24',
    '25 - 34': '25to34',
    '35 - 64': '35to64',
    '65 +': '65plus',
    'Not Stated': 'NotStated'
}

# Create an empty DataFrame to store the transformed data
transformed_df = pd.DataFrame()

# Extract the unique county names
county_names = original_df['employment_titlea'].unique()

# Iterate through each county
for county in county_names:
    county_data = original_df[original_df['employment_titlea'] == county]
    
    # Create a dictionary to store the data for each county
    county_dict = {'County': county}
    
    # Iterate through each age group
    for age_group, prefix in age_groups.items():
        age_group_data = county_data[county_data['Broad_Age_Groups_slash_County'] == age_group]
        
        # Iterate through each column and store the values
        for column in age_group_data.columns:
            county_dict[prefix + '_' + column] = age_group_data[column].values[0]
    
    # Append the county data to the transformed DataFrame
    transformed_df = transformed_df.append(county_dict, ignore_index=True)

# Reset the index
transformed_df.reset_index(drop=True, inplace=True)


In [36]:
transformed_df

,County,Total_employment_titlea,Total_Broad_Age_Groups_slash_County,Total_Broad_Age_Groups_slash_County_original,Total_Total,Total_Working_and_Seeking_Work_slash_No_Work_Available,Total_Working,Total_Seeking_Work_slash_No_Work_Available,Total_Persons_outside_the_Labour_Force,Total_Not_Stated,5to14_employment_titlea,5to14_Broad_Age_Groups_slash_County,5to14_Broad_Age_Groups_slash_County_original,5to14_Total,5to14_Working_and_Seeking_Work_slash_No_Work_Available,5to14_Working,5to14_Seeking_Work_slash_No_Work_Available,5to14_Persons_outside_the_Labour_Force,5to14_Not_Stated,15to17_employment_titlea,15to17_Broad_Age_Groups_slash_County,15to17_Broad_Age_Groups_slash_County_original,15to17_Total,15to17_Working_and_Seeking_Work_slash_No_Work_Available,15to17_Working,15to17_Seeking_Work_slash_No_Work_Available,15to17_Persons_outside_the_Labour_Force,15to17_Not_Stated,18to24_employment_titlea,18to24_Broad_Age_Groups_slash_County,18to24_Broad_Age_Groups_slash_County_original,18to24_Total,18to24_Working_and_Seeking_Work_slash_No_Work_Available,18to24_Working,18to24_Seeking_Work_slash_No_Work_Available,18to24_Persons_outside_the_Labour_Force,18to24_Not_Stated,25to34_employment_titlea,25to34_Broad_Age_Groups_slash_County,25to34_Broad_Age_Groups_slash_County_original,25to34_Total,25to34_Working_and_Seeking_Work_slash_No_Work_Available,25to34_Working,25to34_Seeking_Work_slash_No_Work_Available,25to34_Persons_outside_the_Labour_Force,25to34_Not_Stated,35to64_employment_titlea,35to64_Broad_Age_Groups_slash_County,35to64_Broad_Age_Groups_slash_County_original,35to64_Total,35to64_Working_and_Seeking_Work_slash_No_Work_Available,35to64_Working,35to64_Seeking_Work_slash_No_Work_Available,35to64_Persons_outside_the_Labour_Force,35to64_Not_Stated,65plus_employment_titlea,65plus_Broad_Age_Groups_slash_County,65plus_Broad_Age_Groups_slash_County_original,65plus_Total,65plus_Working_and_Seeking_Work_slash_No_Work_Available,65plus_Working,65plus_Seeking_Work_slash_No_Work_Available,65plus_Persons_outside_the_Labour_Force,65plus_Not_Stated,NotStated_employment_titlea,NotStated_Broad_Age_Groups_slash_County,NotStated_Broad_Age_Groups_slash_County_original,NotStated_Total,NotStated_Working_and_Seeking_Work_slash_No_Work_Available,NotStated_Working,NotStated_Seeking_Work_slash_No_Work_Available,NotStated_Persons_outside_the_Labour_Force,NotStated_Not_Stated
0,KENYA,KENYA,Total,KENYA,"41,235,190","19,677,401 2,621,158","19,677,401","2,621,158","18,927,688","8,943",KENYA,5 - 14,5 - 14,"12,505,987","969,868 -","969,868",-,"11,534,179","1,940",KENYA,15 - 17,15 - 17,"3,360,980","379,497 127,428","379,497","127,428","2,853,492",563,KENYA,18 - 24,18 - 24,"6,288,095","2,891,426 708,167","2,891,426","708,167","2,687,184","1,318",KENYA,25 - 34,25 - 34,"7,333,397","5,544,992 939,317","5,544,992","939,317","847,376","1,712",KENYA,35 - 64,35 - 64,"9,886,139","8,371,755 791,492","8,371,755","791,492","720,494","2,398",KENYA,65 +,65 +,"1,860,021","1,519,863 54,754","1,519,863","54,754","284,963",441,KENYA,Not Stated,Not Stated,571,- -,-,-,-,571
1,MOMBASA,MOMBASA,Total,MOMBASA,"1,043,603","423,439 141,942","423,439","141,942","477,965",257,MOMBASA,5 - 14,5 - 14,"243,191","3,289 -","3,289",-,"239,864",38,MOMBASA,15 - 17,15 - 17,"56,896","2,823 2,632","2,823","2,632","51,432",9,MOMBASA,18 - 24,18 - 24,"179,599","64,289 38,583","64,289","38,583","76,691",36,MOMBASA,25 - 34,25 - 34,"260,491","158,840 53,967","158,840","53,967","47,620",64,MOMBASA,35 - 64,35 - 64,"280,908","186,702 44,595","186,702","44,595","49,532",79,MOMBASA,65 +,65 +,"22,494","7,496 2,165","7,496","2,165","12,826",7,MOMBASA,Not Stated,Not Stated,24,- -,-,-,-,24
2,KWALE,KWALE,Total,KWALE,"728,767","333,587 44,175","333,587","44,175","350,840",165,KWALE,5 - 14,5 - 14,"255,799","24,000 -","24,000",-,"231,756",43,KWALE,15 - 17,15 - 17,"60,598","8,881 1,418","8,881","1,418","50,290",9,KWALE,18 - 24,18 - 24,"106,584","55,440 11,314","55,440","11,314","39,807",23,KWALE,25 - 34,25 - 34,"116,234","89,047 15,303","89

In [38]:
transformed_df.to_csv("df_edu_employment_separated_with_kenya.csv")

In [39]:
df_edu_employment_separated=pd.read_csv("df_edu_employment_separated.csv")